In [1]:
import urllib3
import json
import requests
import io
import pandas as pd
import zipp
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar

In [3]:

path='https://apis.cbs.gov.il/series/catalog/level?id=1&format=json&download=false'
response = requests.get(path)
response.raise_for_status()  # Ensure the request was successful
# archive = zipp.Path(io.BytesIO(response.content))

In [4]:
response

<Response [200]>